In [1]:
import json

from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from opentelemetry.trace import Status, StatusCode

from openinference.instrumentation import TraceConfig
from openinference.instrumentation.config import OITracer
from openinference.semconv.resource import ResourceAttributes

In [2]:
endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={ResourceAttributes.PROJECT_NAME: "openinference-tracer"})
tracer_provider = TracerProvider(resource=resource)
tracer = tracer_provider.get_tracer(__name__)
tracer = OITracer(wrapped=tracer, config=TraceConfig())
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

## Chains

Context manager

In [3]:
with tracer.start_as_current_span(
    "plain-text-mime-type-string-input-string-output",
    openinference_span_kind="chain",
) as span:
    span.set_input("input")
    span.set_output("output")
    span.set_status(Status(StatusCode.OK))

In [4]:
with tracer.start_as_current_span(
    "json-mime-type-string-input-string-output",
    openinference_span_kind="chain",
) as span:
    span.set_input(
        json.dumps({"input-key": "input-value"}),
        mime_type="application/json",
    )
    span.set_output(
        json.dumps({"output-key": "output-value"}),
        mime_type="application/json",
    )
    span.set_status(Status(StatusCode.OK))